In [1]:
from datetime import datetime

import numpy as np
import pandas as pd

In [2]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'AMD', 'AMZN', 'ANET', 'APPH', 'AXON',
    'BROS', 'BYND', 'CHWY', 'COIN', 'COUR', 'CRWD', 'DDOG', 'DKNG',
    'DOCU', 'DUOL', 'EDIT', 'ETSY', 'FLGT', 'FUV', 'FVRR', 'GH', 'GMED',
    'GOOG', 'INTG', 'ISRG', 'JD', 'LSPD', 'LULU','MDB', 'MELI', 'MNST',
    'MSFT', 'MTCH', 'NVCR', 'NVDA', 'NYT', 'OKTA', 'OPEN', 'PANW',
    'PINS', 'PTON', 'QDEL', 'RBLX', 'RDFN', 'RGEN', 'ROKU', 'SE',
    'SOLO', 'SPLK', 'SQ', 'TASK', 'TDOC', 'TEAM', 'TREX', 'TTD', 'TTWO',
    'TWLO', 'TWTR', 'UPST', 'VEEV', 'WK', 'XPEV', 'ZEN', 'ZNGA']

In [3]:
DL = '/Users/damiansp/Downloads'

In [4]:
!ls $DL

Holdings - Damian Satterthwaite-Phillips.csv
Portfolio_Positions_Feb-25-2022.csv
Positions.csv
mw.csv


### E*Trade

In [5]:
DONGMEI = False

In [6]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
ADSK
AMD


In [7]:
if DONGMEI:
    et_df = (
    pd.read_csv(f'{DL}/Positions.csv', index_col=0, skiprows=1)[['Market Value']]
      .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
    pd.read_csv(f'{DL}/Positions.csv', index_col=0, skiprows=1)[['Market Value']]
      .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    #et_df.ET = et_df.ET.apply(lambda x: round(float(x)))
et_df.head()

,ET
ABMD,312.3
ABNB,155.09
AMZN,3075.77
ANET,370.5
BROS,425.25


In [8]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [9]:
check_symbols(et_df)

All clear


In [10]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,0
ABMD,312
ABNB,155
ADSK,0
AMD,0


### Fidelity

In [11]:
TODAY = datetime.now().date()
TODAY
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Feb-25-2022.csv'

In [12]:
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,ROLLOVER IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,789.39,$1.00,$0.00,$789.39,NaN,NaN,NaN,NaN,1.69%,NaN,NaN,Cash
1,218520300,ROLLOVER IRA,ABMD,ABIOMED INC COM USD0.01,3.00,$312.30,+$5.38,$936.90,+$16.14,+1.75%,+$18.12,+1.97%,2.01%,$918.78,$306.26,Cash
2,218520300,ROLLOVER IRA,ABNB,AIRBNB INC COM CL A,8.00,$155.09,-$3.17,$1240.72,-$25.36,-2.01%,-$127.76,-9.34%,2.66%,$1368.48,$171.06,Cash
3,218520300,ROLLOVER IRA,AMD,ADVANCED MICRO DEVICES INC,13.00,$121.06,+$4.45,$1573.78,+$57.85,+3.81%,+$253.37,+19.18%,3.38%,$1320.41,$101.57,Cash
4,218520300,ROLLOVER IRA,ANET,ARISTA NETWORKS INC COM USD0.0001,4.00,$123.50,+$2.64,$494.00,+$10.56,+2.18%,+$12.62,+2.62%,1.06%,$481.38,$120.35,Cash


In [13]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [14]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,ROLLOVER IRA,ABMD,937
2,ROLLOVER IRA,ABNB,1241
3,ROLLOVER IRA,AMD,1574
4,ROLLOVER IRA,ANET,494
5,ROLLOVER IRA,BYND,89


In [15]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['ROLLOVER IRA'].head()

,ROLLOVER IRA
Symbol,
ABMD,937
ABNB,1241
AMD,1574
ANET,494
BYND,89


In [16]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

ROLLOVER IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [17]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA
AAPL,0,0,2143,0
ABMD,312,937,2811,0
ABNB,155,1241,155,0
ADSK,0,0,0,0
AMD,0,1574,0,0


In [18]:
out.to_csv(f'{DL}/clean.csv')

In [19]:
!open $DL/clean.csv

In [20]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

Holdings - Damian Satterthwaite-Phillips.csv
mw.csv


### MarketWatch

In [71]:
!ls $DL

Holdings - Damian Satterthwaite-Phillips.csv


In [72]:
mw = pd.read_csv(f'{DL}/Holdings - Damian Satterthwaite-Phillips.csv')[['Symbol', 'Value']]
mw.index = mw.Symbol
mw.drop('Symbol', axis=1, inplace=True)
mw.Value = mw.Value.apply(convert_value)
mw.head()

,Value
Symbol,
ABMD,3123
AMZN,3076
AXON,1744
BYND,712
CRWD,3998


In [73]:
check_symbols(mw)

All clear


In [74]:
out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
out.head()

,Value
AAPL,0
ABMD,3123
ABNB,0
ADSK,0
AMD,0


In [75]:
out.to_csv(f'{DL}/mw.csv')

In [76]:
!open $DL/mw.csv

In [21]:
!rm $DL/"Holdings - Damian Satterthwaite-Phillips.csv" $DL/mw.csv
!ls $DL

In [91]:
#!rm $DL/*